In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('ggplot')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
from glob import glob

# Assuming data is located in a directory accessible in this path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
from glob import glob

annot = pd.read_parquet('../input/textocr-text-extraction-from-images-dataset/annot.parquet')
imgs = pd.read_parquet('../input/textocr-text-extraction-from-images-dataset/img.parquet')
img_fns = glob('../input/textocr-text-extraction-from-images-dataset/train_val_images/train_images/*')
plt.style.use('ggplot')


In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class Downsample(nn.Module):
    def __init__(self, in_channels, out_channels, apply_batchnorm=True):
        super(Downsample, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1, bias=not apply_batchnorm)
        self.batchnorm = nn.BatchNorm2d(out_channels) if apply_batchnorm else nn.Identity()
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        return self.activation(x)

class Upsample(nn.Module):
    def __init__(self, in_channels, out_channels, apply_dropout=False):
        super(Upsample, self).__init__()
        self.conv = nn.ConvTranspose2d(in_channels, out_channels, 4, stride=2, padding=1, bias=False)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(0.5) if apply_dropout else nn.Identity()
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.dropout(x)
        return self.activation(x)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.down_blocks = nn.ModuleList([
            Downsample(3, 64, apply_batchnorm=False),
            Downsample(64, 128),
            Downsample(128, 256),
            Downsample(256, 512),
            Downsample(512, 512),
            Downsample(512, 512),
            Downsample(512, 512),
            Downsample(512, 512)
        ])
        self.up_blocks = nn.ModuleList([
            Upsample(512, 512, apply_dropout=True),
            Upsample(1024, 512, apply_dropout=True),
            Upsample(1024, 512, apply_dropout=True),
            Upsample(1024, 512),
            Upsample(1024, 256),
            Upsample(512, 128),
            Upsample(256, 64)
        ])
        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        connections = []
        for down in self.down_blocks:
            x = down(x)
            connections.append(x)
        connections = connections[:-1][::-1]
        for up, conn in zip(self.up_blocks, connections):
            x = up(x)
            x = torch.cat([x, conn], 1)
        return self.final(x)

import torch

# Assuming generator and discriminator are instances of nn.Module
generator = Generator()#.to(device)
generator.load_state_dict(torch.load('/kaggle/input/checkpointforcganwithwhite/generator_with_white_epoch_100.pth',map_location=torch.device('cpu')))





In [ ]:

import matplotlib.pyplot as plt


def generate_images(generator, test_input, target, epoch, display=True):
    generator.eval()  # Set the generator to evaluation mode
    with torch.no_grad():
        prediction = generator(test_input).to('cpu')

    test_input = test_input.to('cpu')
    target = target.to('cpu')
    
    # Each image type will have its own column, and images will be displayed in rows
    fig, axs = plt.subplots(2, 3, figsize=(15, 40))  # 10 rows, 3 columns

    for i in range(1):  # Assuming there are exactly 10 images in the batch
        axs[i, 0].imshow(test_input[i].permute(1, 2, 0) * 0.5 + 0.5)
        axs[i, 0].set_title('Input Image')
        axs[i, 0].axis('off')

        axs[i, 1].imshow(target[i].permute(1, 2, 0) * 0.5 + 0.5)
        axs[i, 1].set_title('Real Image')
        axs[i, 1].axis('off')

        axs[i, 2].imshow(prediction[i].permute(1, 2, 0) * 0.5 + 0.5)
        axs[i, 2].set_title('Generated Image')
        axs[i, 2].axis('off')

    plt.suptitle(f'Epoch {epoch}')
    if display:
        plt.show()
    plt.close()

    generator.train()  # Set the generator back to training mode


In [ ]:
import torchvision.transforms.functional as TF

class ResizeMaintainAspectRatio:
    def __init__(self, target_size):
        """
        target_size (int): The target size for the smallest dimension of the image.
        """
        self.target_size = target_size

    def __call__(self, image):
        # Compute the scaling factor to maintain aspect ratio.
        width, height = image.size
        min_dim = min(width, height)
        scaling_factor = self.target_size / min_dim
        
        new_width = max(256,int(width * scaling_factor))
        new_height = max(256,int(height * scaling_factor))

        # Resize the image with the computed dimensions
        image = TF.resize(image, (new_height, new_width))
        return image

class SplitAndCropTransform:
    def __init__(self, output_size, resize_transform=None):
        """
        output_size (tuple): Desired output size of the crop, e.g., (256, 256).
        resize_transform (callable): Transformation to resize the image.
        """
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            self.output_size = output_size
        self.resize_transform = resize_transform

    def __call__(self, image):


        if self.resize_transform:
            image = self.resize_transform(image)

        i, j, h, w = transforms.RandomCrop.get_params(
            image, output_size=self.output_size)

        image = TF.crop(image, i, j, h, w)

        return image

# Example of using the transformations
resize_transform = ResizeMaintainAspectRatio(256)
split_transform = SplitAndCropTransform((256, 256), resize_transform=resize_transform)




In [ ]:
# import torch
# from torchvision import transforms
# from PIL import Image

# transform = transforms.Compose([
#     transforms.ToTensor(),  # Convert the PIL image to a PyTorch tensor
# ])
# for img_path in img_fns:
#     try:
#         image_id = img_path.split('/')[-1].split('.')[0]

#         # Load image
#         image = Image.open(img_path)
#         image = split_transform(image)
#         image_tensor = transform(image)
#         image_tensor = image_tensor.unsqueeze(0)
        
#         # Generate images (replace `image` twice with the actual mask and image if needed)
#         generate_images(generator, image_tensor, image_tensor, 1, display=True)
        
#     except Exception as e:
#         print(f"Error processing {img_path}: {e}")

In [ ]:
def apply_mosaic(image, bbox, pixelation_level=10):
    """
    Applies a mosaic effect to a specified bounding box within an image.
    
    Args:
    image (PIL.Image): The original image.
    bbox (tuple): A tuple of (x, y, width, height) for the bounding box.
    pixelation_level (int): The size of each mosaic block.
    
    Returns:
    PIL.Image: Image with mosaic applied to the region.
    """
    
    
    # Unpack the bounding box
    x, y, w, h = map(int, bbox)
    pixelation_level = max(min(w//3,h//3),1)
#     print("w" + str(w))
#     print("h" + str(h))
    
    # Crop the relevant part of the image
    cropped_img = image.crop((x, y, x + w, y + h))
    
#     print("-1")
    
    # Resize the cropped image to cause the pixelation effect
    cropped_img = cropped_img.resize(
        (w // pixelation_level, h // pixelation_level), 
        resample=Image.BOX
    )
#     print("-2")
    
    # Resize back to original size
    cropped_img = cropped_img.resize(
        (w, h),
        Image.NEAREST
    )
#     print("-3")
    # Paste the pixelated region back to the original image
    image.paste(cropped_img, (x, y))
    
    return image


In [ ]:
from PIL import Image, ImageDraw

def apply_white_blank(image, bbox):
    """
    Applies a white blank to a specified bounding box within an image.

    Args:
    image (PIL.Image): The original image.
    bbox (tuple): A tuple of (x, y, width, height) for the bounding box.

    Returns:
    PIL.Image: Image with a white blank applied to the region.
    """

    # Unpack the bounding box
    x, y, w, h = map(int, bbox)

    # Create a white rectangle
    draw = ImageDraw.Draw(image)
    draw.rectangle([x, y, x + w, y + h], fill="white")

    return image


In [ ]:
# from PIL import Image
# import numpy as np
# import torch

# def draw_random_wavy_lines(image_shape, num_lines=10, thickness=3, max_step=5):
#     print("2")
#     """Generate a mask with wavy lines drawn."""
#     H, W = image_shape
#     mask = np.zeros((H, W), dtype=bool)
#     for _ in range(num_lines):
#         line = random_wavy_line(H, W, max_step, thickness)
#         mask = mask | line
#     return torch.tensor(mask)

# def random_wavy_line(H, W, max_step, thickness):
#     """Generate a wavy line mask with random walk behavior."""
#     print("3")
#     y, x = np.ogrid[:H, :W]
#     line_mask = np.zeros((H, W), dtype=bool)

#     x1, y1 = np.random.randint(0, W), np.random.randint(0, H)

#     for _ in range(100):  # Adjust the number of iterations for longer lines
#         x2 = x1 + np.random.randint(-max_step, max_step)
#         y2 = y1 + np.random.randint(-max_step, max_step)

#         x2 = np.clip(x2, 0, W - 1)
#         y2 = np.clip(y2, 0, H - 1)

#         line_segment = bresenham_line(y1, x1, y2, x2, H, W, thickness)
#         line_mask = line_mask | line_segment

#         x1, y1 = x2, y2

#     return line_mask

# def bresenham_line(y1, x1, y2, x2, H, W, thickness=1):
#     """Generate a mask for a line using Bresenham's line algorithm."""
#     y, x = np.ogrid[:H, :W]
#     line_mask = np.zeros((H, W), dtype=bool)

#     dx = abs(x2 - x1)
#     dy = -abs(y2 - y1)
#     sx = 1 if x1 < x2 else -1
#     sy = 1 if y1 < y2 else -1
#     err = dx + dy

#     while True:
#         brush = (x - x1)**2 + (y - y1)**2 <= (thickness / 2) ** 2
#         line_mask = line_mask | brush

#         if x1 == x2 and y1 == y2:
#             break
#         e2 = 2 * err
#         if e2 >= dy:
#             err += dy
#             x1 += sx
#         if e2 <= dx:
#             err += dx
#             y1 += sy

#     return line_mask

# def apply_white_doodle(image, num_lines=5, thickness=20, max_step=5):
#     print("1")
#     # Open the image and convert it to a NumPy array
#     image_np  = np.array(image)

#     # Create the mask
#     mask = draw_random_wavy_lines(image_np.shape[:2], num_lines, thickness, max_step).numpy()

#     # Apply the mask (assuming the image is in RGB)
#     image_np[mask] = [255, 255, 255]

#     # Convert the modified array back to a PIL image
#     image_modified = Image.fromarray(image_np)

#     return image_modified

# # Example usage:
# # result_image = apply_white_doodle('path/to/your/image.jpg')
# # result_image.show() or result_image.save('output.jpg')


In [ ]:
import numpy as np
import cv2
from PIL import Image

def random_wavy_lines_opencv(H, W, max_step, num_points=50, thickness=2):
    """Generate a wavy line mask using OpenCV."""
    # Initialize the starting point
    x1, y1 = np.random.randint(0, W), np.random.randint(0, H)
    points = [(x1, y1)]

    # Generate a random walk to determine the next points
    for _ in range(num_points):
        x2 = x1 + np.random.randint(-max_step, max_step)
        y2 = y1 + np.random.randint(-max_step, max_step)

        # Keep within image boundaries
        x2 = np.clip(x2, 0, W - 1)
        y2 = np.clip(y2, 0, H - 1)
        points.append((x2, y2))

        x1, y1 = x2, y2

    # Draw the wavy line on a mask
    mask = np.zeros((H, W, 3), dtype=np.uint8)
    cv2.polylines(mask, [np.array(points)], isClosed=False, color=(255, 255, 255), thickness=thickness)

    return mask

def apply_white_doodle_opencv(image, num_lines=10, thickness=2, max_step=15):
    """Apply white doodles on the image using OpenCV."""
    # Open the image and convert it to a NumPy array
    image_np = np.array(image)

    H, W = image_np.shape[:2]
    for _ in range(num_lines):
        mask = random_wavy_lines_opencv(H, W, max_step, thickness=thickness)
        image_np[mask[:, :, 0] == 255] = [255, 255, 255]

    # Convert back to PIL
    image_modified = Image.fromarray(image_np)

    return image_modified

# Example usage:
# image = Image.open('path/to/your/image.jpg')
# result_image = apply_white_doodle_opencv(image)
# result_image.show() or result_image.save('output.jpg')


In [ ]:
import random

def calculate_expanded_bbox(annotations, img_size):
    """
    Calculates a single bounding box that encompasses all specified bounding boxes and expands it.
    
    Args:
    annotations (pd.DataFrame): DataFrame containing 'bbox' columns with format (x, y, w, h).
    img_size (tuple): The size (width, height) of the image.
    
    Returns:
    tuple: A tuple (x, y, w, h) representing the expanded bounding box.
    """
    # Initialize min and max coordinates with extreme values
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = 0, 0
    area = 0
    # Iterate through annotations to find the extreme points
    for _, row in annotations.iterrows():
        x, y, w, h = row['bbox']
        area = area + w*h
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x + w)
        max_y = max(max_y, y + h)
    

    # Calculate the center, width, and height of the bounding box
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    width = max_x - min_x
    height = max_y - min_y
    
    # Expand the box by 2 centered on the original center
    
    new_width = min(img_size[0], 2 * width)
    new_height = min(img_size[1], 2 * height)
    
    # Calculate the new top-left coordinates, ensuring the box stays within the image bounds
    new_x = max(0, center_x - new_width / 2)
    new_y = max(0, center_y - new_height / 2)
    
    # Ensure the bottom-right corner does not go out of bounds
    new_width = min(img_size[0]-new_x, new_width)
    new_height = max(min(img_size[1] - new_y, new_height),256)
    
    if(new_width<256):
        if(new_x<128):
            new_x = 0
        else:
            new_x = new_x - random.randint(0, 128)
        new_width = 256
        
    if(new_height<256):
        if(new_y<128):
            new_y = 0
        else:
            new_y = new_y - random.randint(0, 128)
        new_height = 256
        
    
    if(int(new_x) + int(new_width)>img_size[0] or int(new_y) + int(new_height)>img_size[1]):
        raise ValueError("too large")

    percentage = area/(int(new_width)*int(new_height))
    if(percentage<0.01):
        raise ValueError("too small")
    
    
    return (int(new_x), int(new_y), int(new_width), int(new_height))


In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


def weights_init(init_type='gaussian'):
    def init_fun(m):
        classname = m.__class__.__name__
        if (classname.find('Conv') == 0 or classname.find(
                'Linear') == 0) and hasattr(m, 'weight'):
            if init_type == 'gaussian':
                nn.init.normal_(m.weight, 0.0, 0.02)
            elif init_type == 'xavier':
                nn.init.xavier_normal_(m.weight, gain=math.sqrt(2))
            elif init_type == 'kaiming':
                nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                nn.init.orthogonal_(m.weight, gain=math.sqrt(2))
            elif init_type == 'default':
                pass
            else:
                assert 0, "Unsupported initialization: {}".format(init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.constant_(m.bias, 0.0)

    return init_fun


class VGG16FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        vgg16 = models.vgg16(pretrained=True)
        self.enc_1 = nn.Sequential(*vgg16.features[:5])
        self.enc_2 = nn.Sequential(*vgg16.features[5:10])
        self.enc_3 = nn.Sequential(*vgg16.features[10:17])

        # fix the encoder
        for i in range(3):
            for param in getattr(self, 'enc_{:d}'.format(i + 1)).parameters():
                param.requires_grad = False

    def forward(self, image):
        results = [image]
        for i in range(3):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            results.append(func(results[-1]))
        return results[1:]


class PartialConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super().__init__()
        self.input_conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                                    stride, padding, dilation, groups, bias)
        self.mask_conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                                   stride, padding, dilation, groups, False)
        self.input_conv.apply(weights_init('kaiming'))

        torch.nn.init.constant_(self.mask_conv.weight, 1.0)

        # mask is not updated
        for param in self.mask_conv.parameters():
            param.requires_grad = False

    def forward(self, input, mask):
        # http://masc.cs.gmu.edu/wiki/partialconv
        # C(X) = W^T * X + b, C(0) = b, D(M) = 1 * M + 0 = sum(M)
        # W^T* (M .* X) / sum(M) + b = [C(M .* X) – C(0)] / D(M) + C(0)

        output = self.input_conv(input * mask)
        if self.input_conv.bias is not None:
            output_bias = self.input_conv.bias.view(1, -1, 1, 1).expand_as(
                output)
        else:
            output_bias = torch.zeros_like(output)

        with torch.no_grad():
            output_mask = self.mask_conv(mask)

        no_update_holes = output_mask == 0
        mask_sum = output_mask.masked_fill_(no_update_holes, 1.0)

        output_pre = (output - output_bias) / mask_sum + output_bias
        output = output_pre.masked_fill_(no_update_holes, 0.0)

        new_mask = torch.ones_like(output)
        new_mask = new_mask.masked_fill_(no_update_holes, 0.0)

        return output, new_mask


class PCBActiv(nn.Module):
    def __init__(self, in_ch, out_ch, bn=True, sample='none-3', activ='relu',
                 conv_bias=False):
        super().__init__()
        if sample == 'down-5':
            self.conv = PartialConv(in_ch, out_ch, 5, 2, 2, bias=conv_bias)
        elif sample == 'down-7':
            self.conv = PartialConv(in_ch, out_ch, 7, 2, 3, bias=conv_bias)
        elif sample == 'down-3':
            self.conv = PartialConv(in_ch, out_ch, 3, 2, 1, bias=conv_bias)
        else:
            self.conv = PartialConv(in_ch, out_ch, 3, 1, 1, bias=conv_bias)

        if bn:
            self.bn = nn.BatchNorm2d(out_ch)
        if activ == 'relu':
            self.activation = nn.ReLU()
        elif activ == 'leaky':
            self.activation = nn.LeakyReLU(negative_slope=0.2)

    def forward(self, input, input_mask):
        h, h_mask = self.conv(input, input_mask)
        if hasattr(self, 'bn'):
            h = self.bn(h)
        if hasattr(self, 'activation'):
            h = self.activation(h)
        return h, h_mask


class PConvUNet(nn.Module):
    def __init__(self, layer_size=7, input_channels=3, upsampling_mode='nearest'):
        super().__init__()
        self.freeze_enc_bn = False
        self.upsampling_mode = upsampling_mode
        self.layer_size = layer_size
        self.enc_1 = PCBActiv(input_channels, 64, bn=False, sample='down-7')
        self.enc_2 = PCBActiv(64, 128, sample='down-5')
        self.enc_3 = PCBActiv(128, 256, sample='down-5')
        self.enc_4 = PCBActiv(256, 512, sample='down-3')
        for i in range(4, self.layer_size):
            name = 'enc_{:d}'.format(i + 1)
            setattr(self, name, PCBActiv(512, 512, sample='down-3'))

        for i in range(4, self.layer_size):
            name = 'dec_{:d}'.format(i + 1)
            setattr(self, name, PCBActiv(512 + 512, 512, activ='leaky'))
        self.dec_4 = PCBActiv(512 + 256, 256, activ='leaky')
        self.dec_3 = PCBActiv(256 + 128, 128, activ='leaky')
        self.dec_2 = PCBActiv(128 + 64, 64, activ='leaky')
        self.dec_1 = PCBActiv(64 + input_channels, input_channels,
                              bn=False, activ=None, conv_bias=True)

    def forward(self, input, input_mask):
        h_dict = {}  # for the output of enc_N
        h_mask_dict = {}  # for the output of enc_N

        h_dict['h_0'], h_mask_dict['h_0'] = input, input_mask

        h_key_prev = 'h_0'
        for i in range(1, self.layer_size + 1):
            l_key = 'enc_{:d}'.format(i)
            h_key = 'h_{:d}'.format(i)
            h_dict[h_key], h_mask_dict[h_key] = getattr(self, l_key)(
                h_dict[h_key_prev], h_mask_dict[h_key_prev])
            h_key_prev = h_key

        h_key = 'h_{:d}'.format(self.layer_size)
        h, h_mask = h_dict[h_key], h_mask_dict[h_key]

        # concat upsampled output of h_enc_N-1 and dec_N+1, then do dec_N
        # (exception)
        #                            input         dec_2            dec_1
        #                            h_enc_7       h_enc_8          dec_8

        for i in range(self.layer_size, 0, -1):
            enc_h_key = 'h_{:d}'.format(i - 1)
            dec_l_key = 'dec_{:d}'.format(i)

            h = F.interpolate(h, scale_factor=2, mode=self.upsampling_mode)
            h_mask = F.interpolate(
                h_mask, scale_factor=2, mode='nearest')

            h = torch.cat([h, h_dict[enc_h_key]], dim=1)
            h_mask = torch.cat([h_mask, h_mask_dict[enc_h_key]], dim=1)
            h, h_mask = getattr(self, dec_l_key)(h, h_mask)

        return h, h_mask

    def train(self, mode=True):
        """
        Override the default train() to freeze the BN parameters
        """
        super().train(mode)
        if self.freeze_enc_bn:
            for name, module in self.named_modules():
                if isinstance(module, nn.BatchNorm2d) and 'enc' in name:
                    module.eval()


# if __name__ == '__main__':
#     size = (1, 3, 5, 5)
#     input = torch.ones(size)
#     input_mask = torch.ones(size)
#     input_mask[:, :, 2:, :][:, :, :, 2:] = 0

#     conv = PartialConv(3, 3, 3, 1, 1)
#     l1 = nn.L1Loss()
#     input.requires_grad = True

#     output, output_mask = conv(input, input_mask)
#     loss = l1(output, torch.randn(1, 3, 5, 5))
#     loss.backward()

#     assert (torch.sum(input.grad != input.grad).item() == 0)
#     assert (torch.sum(torch.isnan(conv.input_conv.weight.grad)).item() == 0)
#     assert (torch.sum(torch.isnan(conv.input_conv.bias.grad)).item() == 0)

#     # model = PConvUNet()
#     # output, output_mask = model(input, input_mask)
model = PConvUNet()

In [ ]:
import torch



def load_checkpoint(filename):
    checkpoint = torch.load(filename,map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state'])
    epoch = checkpoint['epoch']
    return epoch

# Load the checkpoint
checkpoint_filename = '/kaggle/input/checkpointforpartinalconvepoch40/checkpointWithWhiteLines_epoch_40.pth'  # Replace X with the actual epoch number
start_epoch = load_checkpoint(checkpoint_filename)

# Now the model and optimizer have the state loaded from the checkpoint
print(f'Checkpoint loaded, starting from epoch {start_epoch}')

In [ ]:
# output_directory = 'cGAN_images_with_white'
# os.makedirs(output_directory, exist_ok=True)

# import torch
# import numpy as np
# import torchvision.transforms as transforms
# from PIL import Image
# import matplotlib.pyplot as plt
# import os


# transform = transforms.Compose([
#     transforms.ToTensor(),  # Convert the PIL image to a PyTorch tensor
# ])

# def tensor_to_pil(tensor):
#     tensor = tensor.detach().cpu()  # Detach from gradients and move to CPU
#     array = tensor.permute(1, 2, 0).numpy()  # Convert to HWC format
#     array = array * 255  # Denormalize
#     array = array.clip(0, 255).astype(np.uint8)  # Clip and convert to uint8
#     return Image.fromarray(array)


# for img_path in img_fns:
#     try:
#         image_id = img_path.split('/')[-1].split('.')[0]

#         # Load image
#         image = Image.open(img_path)

#         # Filter annotations for this image
#         relevant_annots = annot.query('image_id == @image_id')

#         if not relevant_annots.empty:
#             x_c, y_c, w_c, h_c = calculate_expanded_bbox(relevant_annots, image.size)

# #             # Process each bbox
# #             for _, row in relevant_annots.iterrows():
# #                 #image = apply_mosaic(image, row['bbox'])
# #                 image = apply_white_blank(image, row['bbox'])
# #             #image = apply_white_doodle(image)

#             # Crop the mosaic image
#             image = image.crop((x_c, y_c, x_c + w_c, y_c + h_c))
# #             image = split_transform(image)
#             original = image.copy()
            
# #             image = transform(image)
# #             image = image.unsqueeze(0)
# #             with torch.no_grad():
# #                 image = generator(image)

# #             image = image[0]
# #             image = tensor_to_pil(image)
            
#             image = apply_white_doodle_opencv(image)
# #             image = transform(image)
# #             image = image.unsqueeze(0)
# #             with torch.no_grad():
# #                 image = model(image)
# #             image = image[0]
# #             image = tensor_to_pil(image)
            
            


# #             # Crop the original image to the same region
# #             original = Image.open(img_path)
# #             original = original.crop((x_c, y_c, x_c + w_c, y_c + h_c))

#             # Create a new image by combining the original and the processed
#             combined_image = Image.new('RGB', (original.width * 2, original.height))
#             combined_image.paste(original, (0, 0))
#             combined_image.paste(image, (original.width, 0))

# #             # Display the image
# #             plt.imshow(combined_image)
# #             plt.axis('off')
# #             plt.show()

#             # Save the combined image
#             combined_image.save(os.path.join(output_directory, f'{image_id}_combined.jpg'))

#     except Exception as e:
#         print(f"An error occurred while processing {img_path}: {e}")


In [ ]:
generator.eval()

In [ ]:
output_directory = 'cGAN_images_with_white'
os.makedirs(output_directory, exist_ok=True)

import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import os


transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the PIL image to a PyTorch tensor
])

def tensor_to_pil(tensor):
    tensor = tensor.detach().cpu()  # Detach from gradients and move to CPU
    array = tensor.permute(1, 2, 0).numpy()  # Convert to HWC format
    array = array * 255  # Denormalize
    array = array.clip(0, 255).astype(np.uint8)  # Clip and convert to uint8
    return Image.fromarray(array)


for img_path in img_fns:
    try:
        image_id = img_path.split('/')[-1].split('.')[0]

        # Load image
        image = Image.open(img_path)

        # Filter annotations for this image
        relevant_annots = annot.query('image_id == @image_id')

        if not relevant_annots.empty:
            x_c, y_c, w_c, h_c = calculate_expanded_bbox(relevant_annots, image.size)

#             # Process each bbox
#             for _, row in relevant_annots.iterrows():
#                 #image = apply_mosaic(image, row['bbox'])
#                 image = apply_white_blank(image, row['bbox'])
                
            
            # Crop the mosaic image
            image = image.crop((x_c, y_c, x_c + w_c, y_c + h_c))
            
            image = split_transform(image)
            
            original = image.copy()
                

            
            image = transform(image)
#             print(image.shape)
            image = image.unsqueeze(0)
#             print(image.shape)
#             print("2")
            with torch.no_grad():
#                 print("3")
                image = generator(image)
#                 print("0")
                
#             print("1")
            image = image[0]
            
            image = tensor_to_pil(image)

            image = apply_white_doodle_opencv(image)


            

            # Crop the original image to the same region
#             original = Image.open(img_path)
#             original = original.crop((x_c, y_c, x_c + w_c, y_c + h_c))

            # Create a new image by combining the original and the processed
            combined_image = Image.new('RGB', (original.width * 2, original.height))
            combined_image.paste(original, (0, 0))
            combined_image.paste(image, (original.width, 0))

#             Display the image
            plt.imshow(combined_image)
            plt.axis('off')
            plt.show()

            # Save the combined image
            combined_image.save(os.path.join(output_directory, f'{image_id}_combined.jpg'))

    except Exception as e:
        print(f"An error occurred while processing {img_path}: {e}")


In [ ]:
import zipfile
import os

def zip_directory(folder_path, output_path):
    """
    Zips the contents of the specified folder.

    Args:
    folder_path (str): The path to the folder to zip.
    output_path (str): The path where the zip file will be saved.
    """
    # Create a ZipFile object in 'write' mode
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the full path of the file
                full_path = os.path.join(root, file)
                # Add file to zip
                zipf.write(full_path, arcname=os.path.relpath(full_path, folder_path))

output_directory = 'cGAN_images_with_white'
zip_output_path = 'processed_images.zip'

# First, we ensure all processing is done and saved to 'output_directory'
# After processing all images and saving them:
zip_directory(output_directory, zip_output_path)
print(f'All files zipped successfully into {zip_output_path}')
